## TFlite Prediction For Bounding Box

In [ ]:
!pip install -U --no-cache-dir tensorflow==2.5.0
!pip install -U numpy==1.18.5
!pip install -U Pillow==7.2.0
!pip install -U opencv-python==4.5.1.48

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://asia-python.pkg.dev/datature-nexus/python/simple
     |▎                               | 3.4 MB 149 kB/s eta 0:50:23

### Imports

In [1]:
import os
import cv2
import glob
import numpy as np
import tensorflow as tf

from PIL import Image

os.environ["CUDA_VISIBLE_DEVICES"] = "-1" ## Uncomment to use GPU

## Prepare Preparation

### Set all necessary variables

In [2]:
input_folder = "../../input" # Path to folder that contains input images
output_folder = "../../output" # Path to folder to store predicted images
width = 640 # Width of image to load into model
height = 640 # Height of image to load into model
threshold = 0.7 # Prediction confidence threshold
model = "./tf.lite" # Path to exported tensorflow pb model
label = "./label_map.pbtxt" # Path to label map

### Label map loader

In [3]:
def load_label_map(label_map_path):
    label_map = {}

    with open(label_map_path, "r") as label_file:
        for line in label_file:
            if "id" in line:
                label_index = int(line.split(":")[-1])
                label_name = next(label_file).split(":")[-1].strip().strip("'")
                label_map[label_index] = {
                    "id": label_index,
                    "name": label_name,
                }

    return label_map

## Prediction process

### Load label_map, color_map

In [4]:
## Load label map and color map
category_index = load_label_map(label)

color_map = {}
for each_class in range(len(category_index)):
    color_map[each_class] = [
        int(i) for i in np.random.choice(range(256), size=3)
    ]

### Load model into interpreter

In [5]:
interpreter = tf.lite.Interpreter(model_path=model)

### Visualize details of input & output layers 

In [6]:
## Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

## Print output details.
for idx, each_out in enumerate(output_details):
    print("{}: {}\n".format(idx,each_out))

0: {'name': 'StatefulPartitionedCall:5', 'index': 409, 'shape': array([1], dtype=int32), 'shape_signature': array([1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}

1: {'name': 'StatefulPartitionedCall:1', 'index': 497, 'shape': array([1, 1, 1], dtype=int32), 'shape_signature': array([ 1, -1, -1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}

2: {'name': 'StatefulPartitionedCall:2', 'index': 479, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([ 1, -1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': arr

### Redefine expected image size of input layer 

In [7]:
## Current size is set to accept 640 by 640 image
interpreter.resize_tensor_input(input_details[0]['index'], (1, width, height, 3))
interpreter.allocate_tensors()

### Run prediction on each image

In [ ]:
## Run prediction on each image
for each_image in glob.glob(os.path.join(input_folder, "*")):
        print("Predicting for {}...".format(each_image))
        
        ## Load image into numpy array and resize accordingly
        img = cv2.imread(each_image)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        origi_shape = img.shape
        new_img = cv2.resize(img, (width, height))
        
        ## Execute prediction
        interpreter.set_tensor(input_details[0]['index'], [new_img.astype(np.uint8)])
        interpreter.invoke()

        ## Extract data and filter against desired threshold
        for each_layer in output_details:
            if each_layer["name"] == "StatefulPartitionedCall:4":
                scores = np.squeeze(interpreter.get_tensor(each_layer['index']))
                
            if each_layer["name"] == "StatefulPartitionedCall:1":
                bboxes = np.squeeze(interpreter.get_tensor(each_layer['index']))
                
            if each_layer["name"] == "StatefulPartitionedCall:2":
                classes = np.squeeze(interpreter.get_tensor(each_layer['index']))

        ## Filter out predictions below threshold
        indexes = np.where(scores > float(threshold))

        ## Extract predictions
        scores = scores[indexes]
        bboxes = bboxes[indexes]
        classes = classes[indexes]

        ## Draw Predictions
        image_origi = np.array(img)

        if len(bboxes) != 0:
            for idx, each_bbox in enumerate(bboxes):
                color = color_map.get(classes[idx] - 1)

                ## Draw bounding box
                cv2.rectangle(
                    image_origi,
                    (
                        int(each_bbox[1] * origi_shape[1]),
                        int(each_bbox[0] * origi_shape[0]),
                    ),
                    (
                        int(each_bbox[3] * origi_shape[1]),
                        int(each_bbox[2] * origi_shape[0]),
                    ),
                    color,
                    2,
                )

                ## Draw label background
                cv2.rectangle(
                    image_origi,
                    (
                        int(each_bbox[1] * origi_shape[1]),
                        int(each_bbox[2] * origi_shape[0]),
                    ),
                    (
                        int(each_bbox[3] * origi_shape[1]),
                        int(each_bbox[2] * origi_shape[0] + 15),
                    ),
                    color,
                    -1,
                )

                ## Insert label class & score
                cv2.putText(
                    image_origi,
                    "Class: {}, Score: {}".format(
                        str(category_index[classes[idx]]["name"]),
                        str(round(scores[idx], 2)),
                    ),
                    (
                        int(each_bbox[1] * origi_shape[1]),
                        int(each_bbox[2] * origi_shape[0] + 10),
                    ),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.3,
                    (0, 0, 0),
                    1,
                    cv2.LINE_AA,
                )

        ## Save predicted image
        filename = os.path.basename(each_image)
        image_predict = Image.fromarray(image_origi)
        image_predict.save(os.path.join(output_folder, filename))

        print(
            "Saving predicted images to {}...".format(
                os.path.join(OUTPUT, filename)
            )
        )